In [1]:
import pandas as pd
import requests as r
import numpy
import time

def get_headers():
    """Make headers to access url"""
    headers="""accept: application/json
    cid: undefined
    content-type: application/json
    Referer: https://www.walmart.com/m/electronics?page=2&ps=60
    sec-ch-ua: " Not A;Brand";v="99", "Chromium";v="90", "Google Chrome";v="90"
    sec-ch-ua-mobile: ?0
    User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36
    useragent: undefined"""
    headers = dict([i.split(': ') for i in headers.split('\n')])
    return headers

def get_data():
    """Use headers to access URL->loop through url pages->
    resp/json to df-> append each df to df_walmart"""
    df_walmart= pd.DataFrame()
    url1 ="""https://www.walmart.com/cp/api/get-deals-list?prg=desktop&dealsId=electronics&ps=60&page="""
    url2 ="""&shelf_id=5788839&shelfType=manual"""
    for i in range(1,5):
        url = url1+f'{i}'+url2
        response = r.get(url, headers = headers)
        data=response.json()
        data= pd.json_normalize(data['items'])
        df_walmart = df_walmart.append(data)
        time.sleep(5) 
    return df_walmart
    
def save_copy(df_walmart):  
    """Save org df and make a copy to clean"""
    df_walmart.reset_index(drop=True, inplace=True)
    df_walmart.to_csv('.\output\org_walmart.csv', index=False)
    df_copy = df_walmart.copy()
    return df_copy

def clean_df(df_copy):
    """clean copy of df to make clean df"""
    df_copy.drop(columns = ['usItemId', 'shippingPassEligible', 'wfsEnabled', 'pickupDiscountEligible', 'preOrderAvailable', 'virtualPack', 'premiumBrand', 'is_limited_qty', 'isLimitedQuantity','visibleSwatches','primaryOffer.offerId', 'fulfillment.isS2H', 'fulfillment.isS2S','fulfillment.isSOI', 'fulfillment.isPUT', 'fulfillment.s2SDisplayFlags','ppu.unit', 'ppu.amount', 'ppu.currencyCode','mediaLanguage', 'sellerBadge','variants.variantMeta', 'variants.variantData', 'isbn', 'mediaFormat','primaryOffer.minPrice', 'primaryOffer.maxPrice','primaryOffer.showMinMaxPrice', 'preOrderAvailableDate', 'launchDate','submapType', 'bundleType', 'primaryOffer.installmentFrequency','primaryOffer.durationWithInterest.duration','primaryOffer.durationWithInterest.durationUOM','primaryOffer.durationWithInterest.interest','primaryOffer.durationWithInterest.interestUOM','inventory.displayFlags', 'fulfillment.thresholdAmount','fulfillment.thresholdCurrencyCode', 'mediaRating','primaryOffer.unitPriceDisplayCondition', 'consoleKstem','minItemCountPerOrder', 'mediaRunningTime','geoItemClassification', 'giftingEligible', 'pcsType','wmtgPricePerUnitQuantity',  'isHeartable','countPerPack', 'isWMPlusShipping', 'blitzItem', 'marketPlaceItem','twoDayShippingEligible', 'specialOfferBadge', 'specialOfferText','specialOfferLink', 'sellerId', 'sellerName', 'enableAddToCart','canAddToCart', 'cta', 'showPriceAsAvailable', 'itemClassId', 'primaryOffer.currencyCode', 'primaryOffer.showWasPrice', 'inventory.availableOnline', 'productPageUrl', 'fulfillment.s2HDisplayFlags','productType', 'imageProps.src', 'imageProps.srcSet', 'seeAllLink', 'department','seeAllName'], inplace=True)
    df_copy = df_copy.loc[:, ['productId', 'primaryOffer.savingsAmount', 'primaryOffer.offerPrice', 'primaryOffer.listPrice', 'title', 'description', 'productCategory', 'customerRating', 'numReviews', 'brand', 'initialQuantity', 'quantity','standardUpc','imageUrl']]
    df_copy['description'] = df_copy['description'].str.replace('(<li>)', '').str.replace('(</li>)', '').str.replace('<sup>','').str.replace('</sup>','').str.replace('<b>','').str.replace('</b>','').str.replace('<div>','').str.replace('</div>','').str.replace('<p>','').str.replace('</p>','').str.replace('p class="msonormal">', '')
    df_copy['description']= df_copy['description'].str.replace('(<.*>)', '')
    df_copy['description']=df_copy['description'].fillna(value='')
    cols_rename = {'primaryOffer.savingsAmount': 'Savings$' ,'primaryOffer.offerPrice': 'Sale_price$', 'primaryOffer.listPrice': 'Orig_price$', 'title':'Item'}
    df_copy.rename(columns = cols_rename, inplace=True)
    df_copy.dropna(inplace=True)
    df = df_copy
    return df

def save_df(df):
    """save final df"""
    df.to_csv('.\output\walmart.csv', index=False)
    
if __name__ == '__main__':
    df_walmart = get_data()
    df_copy = save_copy(df_walmart)
    df = clean_df(df_copy)
    save_df(df)

NameError: name 'headers' is not defined

In [98]:
pd.set_option('display.max_rows', None, 'display.max_columns', None)

In [100]:
df

,productId,Savings$,Sale_price$,Orig_price$,Item,description,productCategory,customerRating,numReviews,brand,initialQuantity,quantity,standardUpc,imageUrl
0,3JMS5HDQJKWL,22.00,197.00,219.00,Apple AirPods Pro,Active Noise Cancellation for immersive soundT...,Headphones,4.6,5678,[Apple],45860,45860,[00190199246850],https://i5.walmartimages.com/asr/c67bebef-646f...
2,7GPRRVEITAG5,40.00,249.00,289.00,"HP 14"" Pentium 4GB/64GB Chromebook, 14"" HD Dis...",Chrome with integrated Intel UHD Graphics 600 ...,Laptop Computers,4.2,357,[HP],13332,13332,[00194721548283],https://i5.walmartimages.com/asr/617aef72-8c6d...
3,2RRKOAAOOFW4,30.00,169.00,199.00,Apple Watch Series 3 GPS - 38mm - Sport Band -...,Elevated in every way.Space Gray aluminum case...,Smart Watches,4.6,4770,[Apple],9657,9657,[00190198806116],https://i5.walmartimages.com/asr/83ed7cf5-45b5...
9,1F58BAASE008,30.00,199.00,229.00,Apple Watch Series 3 GPS - 42mm - Sport Band -...,GPS and a barometric altimeter track how far a...,Smart Watches,4.7,1571,[Apple],18864,18864,[00190198806925],https://i5.walmartimages.com/asr/64985038-90f0...
11,6O1V22CEVP7M,51.00,148.95,199.95,Fitbit Versa 2 Smartwatch,Built-in voice controlSleep score in app24/7 h...,Smart Watches,4.3,559,[Fitbit],2297,2297,[00811138036690],https://i5.walmartimages.com/asr/6e5b77fa-3304...
12,6OVC72F7R5QF,66.99,133.00,199.99,"TCL 32"" Class 720P HD LED Roku Smart TV 3 Seri...","TCL 32"" Class HD (720P) Roku Smart LED TV (32S...",Televisions,4.3,2063,[TCL],10053,10053,[00846042013543],https://i5.walmartimages.com/asr/fe7464f9-9701...
13,73A86BAYS6RT,50.00,349.00,399.00,"Straight Talk Apple iPhone XR, 64GB - Prepaid ...",ST/TW iPhone XR Black,Cell Phones,4.4,1556,[Apple],3159,3159,[00194252140116],https://i5.walmartimages.com/asr/facbb465-6b0c...
14,5ZQZF0RUAKRN,330.00,369.00,699.00,"Lenovo IdeaPad Flex 5 14"" FHD Convertible, AMD...","Lenovo IdeaPad Flex 52-in-1 Laptop14"" FHD Touc...",Laptop Computers,4.3,536,[Lenovo],2011,2011,[00194632976182],https://i5.walmartimages.com/asr/b163a876-4363...
17,34HTKPCNI7NB,30.00,99.00,129.00,"Straight Talk LG Stylo 5, 32GB, Black - Prepai...","6.2"" FHD+ Touchscreen DisplayAndroid 9.0, Pie1...",Cell Phones,3.7,701,[LG],1186,1186,[00616960331452],https://i5.walmartimages.com/asr/8775683a-e21b...
19,47FG5RLZAG48,310.00,439.00,749.00,"Gateway 15.6"" FHD Ultra Slim Notebook, Intel C...",Windows 10 Home10thGen Intel Core i5-1035G1 Pr...,Laptop Computers,3.9,365,[Gateway],18138,18138,[00812550032970],https://i5.walmartimages.com/asr/d4363192-ff9f...
